In [1]:
import pandas as pd

ANALYSIS ON IDS

In [92]:
#import IDS data
IDS_data = pd.read_csv(r'C:\Users\user\Desktop\MOPA Reconciliation\Gisagara &Nyanza\Season Clients Detailed_20200923-102226.csv')


In [99]:
#Preliminary info on IDS data on subsidized inputs
GovLocationParent = 'Mukingo'
SectorDf = IDS_data [IDS_data ['GovLocationParent']==GovLocationParent ]
SectorClients = SectorDf.groupby('GovLocationParent')['OAFID'].count()

SubsInputsdf = SectorDf[['DistrictName','SectorName','SiteName','LastName','FirstName','OAFID','NationalID','2021A_DAP kg','2021A_NPK 17 kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 403 kg','2021A_SC 637 kg','2021A_UREA kg','2021A_WH 403 kg','2021A_WH 505 kg','2021A_WH 605 kg']]

SubsInputsMelt = SubsInputsdf.melt(['DistrictName','SectorName','SiteName','LastName','FirstName','OAFID','NationalID'],['2021A_DAP kg','2021A_NPK 17 kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 403 kg','2021A_SC 637 kg','2021A_UREA kg','2021A_WH 403 kg','2021A_WH 505 kg','2021A_WH 605 kg'], 'inputName','Quantity').fillna(0)

#Cleaning some IDS columns
SubsInputsMelt['NationalID'] =SubsInputsMelt['NationalID'].astype(str).replace('\.0', '', regex=True)

IDSInputsTxns = SubsInputsMelt[SubsInputsMelt['Quantity']>0]
IDSInputsTxns ['inputName'] = IDSInputsTxns ['inputName'].replace(['2021A_DAP kg','2021A_NPK 17 kg','2021A_UREA kg','2021A_WH 403 kg','2021A_SC 403 kg','2021A_Pan 53 kg'],['DAP','NPK','UREA','WH403','SC403','PAN53'])
IDSInputsTxns =IDSInputsTxns.rename(columns={'inputName':'Product'})
IDSInputsTxns['ProductID'] = IDSInputsTxns['NationalID'] + IDSInputsTxns['Product']

IDSClientsPerInput = IDSInputsTxns.groupby('Product')['OAFID'].count().reset_index()
IDSQtyPerInput = IDSInputsTxns.groupby('Product')['Quantity'].sum().reset_index()
SectorSummary = pd.merge(IDSClientsPerInput,IDSQtyPerInput,on='Product',how='outer').rename(columns={'OAFID':'#Clients'})


SectorSummary 

,Product,#Clients,Quantity
0,DAP,1734,16040.0
1,NPK,611,10960.0
2,PAN53,1716,5070.0
3,UREA,2045,20425.5
4,WH403,1,2.0


In [100]:
#Other IDS data
TotalIDSClients = SubsInputsdf['NationalID'].nunique()

#Transactions
IDSTotalTxn = IDSInputsTxns['OAFID'].count()
SubsInputsMelt.to_excel('SubsInputsMelt.xlsx')

ANALYSIS ON MOPA

In [101]:
#import MOPA data
MOPA = pd.read_excel(r'C:\Users\user\Desktop\MOPA Reconciliation\Gisagara &Nyanza\Mukingo.xlsx')

In [103]:
#Cleaning MOPA data
ColSplit = MOPA ['Requested Qty'].str.split(" ", n = 2, expand = True)
MOPA['Requested_qty'] = ColSplit[0].astype(float)
MOPA['Paid_qty'] = MOPA['Paid Qty'].str.extract('(\d+)').astype(float)
MOPA['Product'] = MOPA['Product'].str.upper()
MOPA = MOPA.rename(columns={'National Id':'NationalID'})
MOPA['NationalID'] = MOPA['NationalID'].astype(str)
MOPA['ProductID'] = MOPA['NationalID'] + MOPA['Product']
MOPA_dataCols = MOPA[['Name','NationalID','Product','Requested_qty','Paid_qty']]

In [104]:
#Preliminary info on IDS data on subsidized inputs

ClientsqtyPerInput =MOPA_dataCols.groupby('Product')['Requested_qty','Paid_qty'].sum().reset_index()
ClientsNumPerInput = MOPA_dataCols.groupby('Product')['NationalID'].count().reset_index()

#Number of clients who took each input in mopa and the total quantity taken per each input
SummaryMOPA = pd.merge(ClientsqtyPerInput,ClientsNumPerInput,on='Product', how='outer').rename(columns={'NationalID':'#Clients'})
SummaryMOPA['Product'] = SummaryMOPA['Product'].replace(['NPK 17:17:17'],['NPK'])
SummaryMOPA

,Product,Requested_qty,Paid_qty,#Clients
0,DAP,34415.00,13698.0,1489
1,NPK,26630.00,8465.0,438
2,PAN53,141.25,112.0,36
3,UREA,47862.60,16837.0,1703


In [105]:
#Other data on IDS
TotalMOPAClients = MOPA['NationalID'].nunique()
#Transactions
TotalMOPATxn = MOPA['NationalID'].count()

MATCHING IDS TO MOPA 

In [106]:
 MOPAvsIDS = pd.merge(SectorSummary,SummaryMOPA,on='Product',how='outer')
 MOPAvsIDS['Difference In #Clients'] = MOPAvsIDS['#Clients_x'] - MOPAvsIDS['#Clients_y']
 MOPAvsIDS['Difference In qty'] = MOPAvsIDS['Quantity'] - MOPAvsIDS['Paid_qty']
 MOPAvsIDS

,Product,#Clients_x,Quantity,Requested_qty,Paid_qty,#Clients_y,Difference In #Clients,Difference In qty
0,DAP,1734,16040.0,34415.00,13698.0,1489.0,245.0,2342.0
1,NPK,611,10960.0,26630.00,8465.0,438.0,173.0,2495.0
2,PAN53,1716,5070.0,141.25,112.0,36.0,1680.0,4958.0
3,UREA,2045,20425.5,47862.60,16837.0,1703.0,342.0,3588.5
4,WH403,1,2.0,NaN,NaN,NaN,NaN,NaN


CLIENT LEVEL CHECK

In [107]:
MOPAInputsTxns = MOPA[['Name', 'Surname', 'NationalID','Product','Paid_qty','ProductID']]
IDSvsMOPA_Clients = pd.merge(IDSInputsTxns,MOPAInputsTxns,on='ProductID', how='left')
IDSvsMOPA_Clients['Match'] = IDSvsMOPA_Clients['Quantity'] == IDSvsMOPA_Clients['Paid_qty']
IDSvsMOPA_Clients.to_excel('IDSVSMOPA.xlsx')

In [108]:
#clients
TotalClientsOnMOPA_IDS = IDSvsMOPA_Clients['OAFID'].nunique()

#Transactions
MatchingTxn = IDSvsMOPA_Clients[IDSvsMOPA_Clients['Match']==True]['OAFID'].count()
NotMatchingTXn = IDSvsMOPA_Clients[IDSvsMOPA_Clients['Match']==False]['OAFID'].count()
OnIDSNotInMOPA = IDSvsMOPA_Clients[IDSvsMOPA_Clients['Paid_qty'].isna()]

In [110]:
#Print clients
print('Total clients on IDS:',TotalIDSClients)
print('Total clients in MOPA:',TotalMOPAClients)
print('Total clients on IDS_MOPA df:',TotalClientsOnMOPA_IDS )


#Transactions
print('Total Txn on IDS:',IDSTotalTxn)
print('Total Txn in MOPA:',TotalMOPATxn)
print('Total Txn IDS match MOPA:',MatchingTxn)
print('Total Txn IDS dont match MOPA:',NotMatchingTXn)

Total clients on IDS: 2191
Total clients in MOPA: 1743
Total clients on IDS_MOPA df: 2079
Total Txn on IDS: 6107
Total Txn in MOPA: 3666
Total Txn IDS match MOPA: 2147
Total Txn IDS dont match MOPA: 3960
